# Post Processing

In [ ]:
import gzip
import pickle
from yaml import Loader, Dumper
import yaml

In [ ]:
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
from coffea import hist
from plots.helpers import makePlot2
from tools.helpers import cutflow_scale_and_merge, getCutFlowTable
from yahist import Hist1D, Hist2D

## Import Histograms

In [ ]:
path = '../analysis/histos/limits_3000.pkl.gz'
with gzip.open(path) as fin:
    scaled_output = pickle.load(fin)

# Data Cards

In [ ]:
MT_vs_sdmass = {
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000': {},
    '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250': {},
    'ZJetsToNuNu_HT':        {},
    'TT':        {},
    'WJetsToLNu_Njet':        {},
    'QCD_bEnriched_HT':        {},
}

variations = [
    'central',
    'up',
    'down',
    'central_0b_up',
    'central_0b_down',
    'central_1b_up',
    'central_1b_down',
    'central_2b_up',
    'central_2b_down',
    'central_1h_up',
    'central_1h_down',
]

JES_unc = dict.fromkeys(
    [
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250',
        'ZJetsToNuNu_HT',
        'TT',
        'WJetsToLNu_Njet',
        'QCD_bEnriched_HT'
    ]
)

b0_unc = dict.fromkeys(
    [
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250',
        'ZJetsToNuNu_HT',
        'TT',
        'WJetsToLNu_Njet',
        'QCD_bEnriched_HT'
    ]
)

b1_unc = dict.fromkeys(
    [
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250',
        'ZJetsToNuNu_HT',
        'TT',
        'WJetsToLNu_Njet',
        'QCD_bEnriched_HT'
    ]
)

b2_unc = dict.fromkeys(
    [
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250',
        'ZJetsToNuNu_HT',
        'TT',
        'WJetsToLNu_Njet',
        'QCD_bEnriched_HT'
    ]
)

h1_unc = dict.fromkeys(
    [
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000',
        '2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250',
        'ZJetsToNuNu_HT',
        'TT',
        'WJetsToLNu_Njet',
        'QCD_bEnriched_HT'
    ]
)

In [ ]:
for sample in MT_vs_sdmass:
    for var in variations:
        MT_vs_sdmass[sample][var] = Hist2D.from_bincounts(
            scaled_output['MT_vs_sdmass_'+var].values(overflow='over')[(sample,)].T,
            (scaled_output['MT_vs_sdmass_'+var].axis('mt').edges(overflow='over'), scaled_output['MT_vs_sdmass_'+var].axis('mass').edges(overflow='over')),
            errors = np.sqrt(scaled_output['MT_vs_sdmass_'+var].values(sumw2=True, overflow='over')[(sample,)][1].T),
        )
        for b in range(0,4):
            for j in range(0,10):
                if round(MT_vs_sdmass[sample][var].counts[b][j], 3) == 0:
                    MT_vs_sdmass[sample][var].counts[b][j] = 0.01
                if round(MT_vs_sdmass[sample][var].errors[b][j], 3) == 0:
                    MT_vs_sdmass[sample][var].errors[b][j] = 0.01
                    
    JES_unc[sample] = abs((MT_vs_sdmass[sample]['up']-MT_vs_sdmass[sample]['down']).counts)/(2*MT_vs_sdmass[sample]['central'].counts)+1
    b0_unc[sample]  = abs((MT_vs_sdmass[sample]['central_0b_up']-MT_vs_sdmass[sample]['central_0b_down']).counts)/(2*MT_vs_sdmass[sample]['central'].counts)+1
    b1_unc[sample]  = abs((MT_vs_sdmass[sample]['central_1b_up']-MT_vs_sdmass[sample]['central_1b_down']).counts)/(2*MT_vs_sdmass[sample]['central'].counts)+1
    b2_unc[sample]  = abs((MT_vs_sdmass[sample]['central_2b_up']-MT_vs_sdmass[sample]['central_2b_down']).counts)/(2*MT_vs_sdmass[sample]['central'].counts)+1
    h1_unc[sample]  = abs((MT_vs_sdmass[sample]['central_1h_up']-MT_vs_sdmass[sample]['central_1h_down']).counts)/(2*MT_vs_sdmass[sample]['central'].counts)+1

## datacards

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500']['central'],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT']['central'],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet']['central'],
            'tt': MT_vs_sdmass['TT']['central'],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['central']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j] + MT_vs_sdmass['TT']['central'].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_1500_150_2.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250']['central'],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT']['central'],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet']['central'],
            'tt': MT_vs_sdmass['TT']['central'],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['central']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j] + MT_vs_sdmass['TT']['central'].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_1250_750_3.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500']['central'],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT']['central'],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet']['central'],
            'tt': MT_vs_sdmass['TT']['central'],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['central']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j] + MT_vs_sdmass['TT']['central'].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_1500_750_3.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600']['central'],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT']['central'],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet']['central'],
            'tt': MT_vs_sdmass['TT']['central'],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['central']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_750_MH2_1600_MHC_1600'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j] + MT_vs_sdmass['TT']['central'].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_1600_750_3.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750']['central'],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT']['central'],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet']['central'],
            'tt': MT_vs_sdmass['TT']['central'],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['central']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j] + MT_vs_sdmass['TT']['central'].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_1750_750_3.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900']['central'],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT']['central'],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet']['central'],
            'tt': MT_vs_sdmass['TT']['central'],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['central']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_750_MH2_1900_MHC_1900'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j] + MT_vs_sdmass['TT']['central'].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_1900_750_3.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000']['central'],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT']['central'],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet']['central'],
            'tt': MT_vs_sdmass['TT']['central'],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['central']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j] + MT_vs_sdmass['TT']['central'].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_2000_750_3.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250']['central'],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT']['central'],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet']['central'],
            'tt': MT_vs_sdmass['TT']['central'],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['central']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_750_MH2_2250_MHC_2250'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j] + MT_vs_sdmass['TT']['central'].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_2250_750_3.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250']['central'],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT']['central'],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet']['central'],
            'tt': MT_vs_sdmass['TT']['central'],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['central']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_250_MH2_1250_MHC_1250'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j] + MT_vs_sdmass['TT']['central'].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_1250_250_3.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500']['central'],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT']['central'],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet']['central'],
            'tt': MT_vs_sdmass['TT']['central'],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['central']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j] + MT_vs_sdmass['TT']['central'].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_1500_250_3.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600']['central'],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT']['central'],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet']['central'],
            'tt': MT_vs_sdmass['TT']['central'],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['central']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1600_MH4_250_MH2_1600_MHC_1600'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j] + MT_vs_sdmass['TT']['central'].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_1600_250_3.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750']['central'],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT']['central'],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet']['central'],
            'tt': MT_vs_sdmass['TT']['central'],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['central']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_250_MH2_1750_MHC_1750'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j] + MT_vs_sdmass['TT']['central'].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_1750_250_3.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900']['central'],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT']['central'],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet']['central'],
            'tt': MT_vs_sdmass['TT']['central'],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['central']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1900_MH4_250_MH2_1900_MHC_1900'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j] + MT_vs_sdmass['TT']['central'].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_1900_250_3.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000']['central'],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT']['central'],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet']['central'],
            'tt': MT_vs_sdmass['TT']['central'],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['central']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j] + MT_vs_sdmass['TT']['central'].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_2000_250_3.txt')

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')
c.addUncertainty('signal_unc', 'lnN')
c.addUncertainty('QCD_unc', 'lnN')
c.addUncertainty('Wjets_unc', 'lnN')
c.addUncertainty('Zinv_unc', 'lnN')
c.addUncertainty('tt_unc', 'lnN')
c.addUncertainty('JES',        'lnN')
c.addUncertainty('0b',         'lnN')
c.addUncertainty('1b',         'lnN')
c.addUncertainty('2b',         'lnN')
c.addUncertainty('1h',         'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['Wjets', 'QCD', 'tt', 'Zjets'], Binname) # signal is automatically added
        
        processes = {
            'signal': MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250']['central'],
            'QCD': MT_vs_sdmass['QCD_bEnriched_HT']['central'],
            'Wjets': MT_vs_sdmass['WJetsToLNu_Njet']['central'],
            'tt': MT_vs_sdmass['TT']['central'],
            'Zjets': MT_vs_sdmass['ZJetsToNuNu_HT']['central']}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'tt',    round(MT_vs_sdmass['TT']['central'].counts[b][j], 3))
        c.specifyExpectation(binname, 'Zjets',    round(MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        c.specifyUncertainty('signal_unc', binname, 'signal', 1.20)
        c.specifyUncertainty('QCD_unc',    binname, 'QCD',   1.50)
        c.specifyUncertainty('Wjets_unc',    binname, 'Wjets',   1.25)
        c.specifyUncertainty('Zinv_unc',   binname, 'Zjets',   1.25)
        c.specifyUncertainty('tt_unc',   binname, 'tt',   1.25)
        
        c.specifyUncertainty('JES', binname, 'signal', JES_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250'][b][j])
        c.specifyUncertainty('JES', binname, 'QCD',   JES_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('JES', binname, 'Wjets',   JES_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('JES', binname, 'tt',  JES_unc['TT'][b][j])
        c.specifyUncertainty('JES', binname, 'Zjets',  JES_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('0b', binname, 'signal', b0_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250'][b][j])
        c.specifyUncertainty('0b', binname, 'QCD',   b0_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('0b', binname, 'Wjets',   b0_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('0b', binname, 'tt',  b0_unc['TT'][b][j])
        c.specifyUncertainty('0b', binname, 'Zjets',  b0_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1b', binname, 'signal', b1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_250_MH2_2000_MHC_2000'][b][j])
        c.specifyUncertainty('1b', binname, 'QCD',   b1_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('1b', binname, 'Wjets',   b1_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('1b', binname, 'tt',  b1_unc['TT'][b][j])
        c.specifyUncertainty('1b', binname, 'Zjets',  b1_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('2b', binname, 'signal', b2_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250'][b][j])
        c.specifyUncertainty('2b', binname, 'QCD',   b2_unc['QCD_bEnriched_HT'][b][j])
        c.specifyUncertainty('2b', binname, 'Wjets',   b2_unc['WJetsToLNu_Njet'][b][j])
        c.specifyUncertainty('2b', binname, 'tt',  b2_unc['TT'][b][j])
        c.specifyUncertainty('2b', binname, 'Zjets',  b2_unc['ZJetsToNuNu_HT'][b][j])
        
        c.specifyUncertainty('1h', binname, 'signal', h1_unc['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2250_MH4_250_MH2_2250_MHC_2250'][b][j])
        
        c.specifyObservation(binname, round(MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts[b][j] + MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts[b][j] + MT_vs_sdmass['TT']['central'].counts[b][j] + MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts[b][j], 3))

        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_2250_250_3.txt')

## Plotting Regions

In [ ]:
bin_array_signal = np.ones(40)
i=0
for row in MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750']['central'].counts:
    for col in row:
        bin_array_signal[i] = col
        i+=1

In [ ]:
bin_array_signal_errors = np.ones(40)
i=0
for row in MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750']['central'].errors:
    for col in row:
        bin_array_signal_errors[i] = col
        i+=1

In [ ]:
bin_array_Z = np.ones(40)
i=0
for row in MT_vs_sdmass['ZJetsToNuNu_HT']['central'].counts:
    for col in row:
        bin_array_Z[i] = col
        i+=1

In [ ]:
bin_array_Z_errors = np.ones(40)
i=0
for row in MT_vs_sdmass['ZJetsToNuNu_HT']['central'].errors:
    for col in row:
        bin_array_Z_errors[i] = col
        i+=1

In [ ]:
bin_array_tt = np.ones(40)
i=0
for row in MT_vs_sdmass['TT']['central'].counts:
    for col in row:
        bin_array_tt[i] = col
        i+=1

In [ ]:
bin_array_tt_errors = np.ones(40)
i=0
for row in MT_vs_sdmass['TT']['central'].errors:
    for col in row:
        bin_array_tt_errors[i] = col
        i+=1

In [ ]:
bin_array_W = np.ones(40)
i=0
for row in MT_vs_sdmass['WJetsToLNu_Njet']['central'].counts:
    for col in row:
        bin_array_W[i] = col
        i+=1

In [ ]:
bin_array_W_errors = np.ones(40)
i=0
for row in MT_vs_sdmass['WJetsToLNu_Njet']['central'].errors:
    for col in row:
        bin_array_W_errors[i] = col
        i+=1

In [ ]:
bin_array_QCD = np.ones(40)
i=0
for row in MT_vs_sdmass['QCD_bEnriched_HT']['central'].counts:
    for col in row:
        bin_array_QCD[i] = col
        i+=1

In [ ]:
bin_array_QCD_errors = np.ones(40)
i=0
for row in MT_vs_sdmass['QCD_bEnriched_HT']['central'].errors:
    for col in row:
        bin_array_QCD_errors[i] = col
        i+=1

In [ ]:
unrolled = [bin_array_Z,  bin_array_QCD, bin_array_tt, bin_array_W,]
unrolled_errors = [bin_array_Z_errors, bin_array_QCD_errors, bin_array_tt_errors, bin_array_W_errors]

labels =[r'$ZJets\to\nu\nu$', r'$QCD\ b-enriched$', r'$t\bar{t}$', r'$WJets\to L\nu$', ]

colors =['#7DDC61', '#C061DC',  '#FE3F6B',  '#FED23F', ]

fig, (ax) = plt.subplots(figsize=(10,10))
hep.cms.label(
    'Preliminary',
    loc=0,
    ax=ax,
    #lumi = 3000,
    rlabel = '(14 TeV)',
    )

hep.histplot(
    [bin_array for bin_array in unrolled],
    #w2=[bin_array**2 for bin_array in unrolled_errors],
    histtype="fill",
    #density = True,
    stack=True,
    label=[label for label in labels],
    color=[color for color in colors],
    ax=ax
    )

hep.histplot(
    bin_array_signal,
    w2=bin_array_signal_errors**2,
    histtype="step",
    #density = True,
    stack=False,
    label='2HDMa_1750_750_10',
    #color=[colors[sample] for sample in backgrounds],
    ax=ax
    )

ax.set_xlabel(r'$Bin$')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
ax.legend(prop={'size': 10})

fig.savefig('/home/users/ewallace/public_html/HbbMET/plots/signal_region/unrolled_stack_1750_750.png')

In [ ]:
bin_array_signal2 = np.ones(40)
k=0
for i in range(len(MT_vs_sdmass_2000_750_central.counts[0])):
    for j in range(len(MT_vs_sdmass_2000_750_central.counts[:,i:i+1])):
        bin_array_signal2[k] = MT_vs_sdmass_2000_750_central.counts[j][i]
        k+=1

In [ ]:
bin_array_signal_errors2 = np.ones(40)
k=0
for i in range(len(MT_vs_sdmass_2000_750_central.errors[0])):
    for j in range(len(MT_vs_sdmass_2000_750_central.errors[:,i:i+1])):
        bin_array_signal_errors2[k] = MT_vs_sdmass_2000_750_central.errors[j][i]
        k+=1

In [ ]:
bin_array_Z2 = np.ones(40)
k=0
for i in range(len(MT_vs_sdmass_Z_central.counts[0])):
    for j in range(len(MT_vs_sdmass_Z_central.counts[:,i:i+1])):
        bin_array_Z2[k] = MT_vs_sdmass_Z_central.counts[j][i]
        k+=1

In [ ]:
bin_array_tt2 = np.ones(40)
k=0
for i in range(len(MT_vs_sdmass_tt_central.counts[0])):
    for j in range(len(MT_vs_sdmass_tt_central.counts[:,i:i+1])):
        bin_array_tt2[k] = MT_vs_sdmass_tt_central.counts[j][i]
        k+=1

In [ ]:
bin_array_W2 = np.ones(40)
k=0
for i in range(len(MT_vs_sdmass_W_central.counts[0])):
    for j in range(len(MT_vs_sdmass_W_central.counts[:,i:i+1])):
        bin_array_W2[k] = MT_vs_sdmass_W_central.counts[j][i]
        k+=1

In [ ]:
bin_array_QCD2 = np.ones(40)
i=0
k=0
for i in range(len(MT_vs_sdmass_QCD_central.counts[0])):
    for j in range(len(MT_vs_sdmass_QCD_central.counts[:,i:i+1])):
        bin_array_QCD2[k] = MT_vs_sdmass_QCD_central.counts[j][i]
        k+=1

In [ ]:
unrolled = [bin_array_Z2,  bin_array_QCD2, bin_array_tt2, bin_array_W2,]
#unrolled_errors = [bin_array_Z_errors, bin_array_QCD_errors, bin_array_tt_errors, bin_array_W_errors]

labels =[r'$ZJets\to\nu\nu$', r'$QCD\ b-enriched$', r'$t\bar{t}$', r'$WJets\to L\nu$', ]

colors =['#7DDC61', '#C061DC',  '#FE3F6B',  '#FED23F', ]

fig, (ax) = plt.subplots(figsize=(10,10))
hep.cms.label(
    'Preliminary',
    loc=0,
    ax=ax,
    #lumi = 3000,
    rlabel = '(14 TeV)',
    )

hep.histplot(
    [bin_array for bin_array in unrolled],
    #w2=[bin_array**2 for bin_array in unrolled_errors],
    histtype="fill",
    #density = True,
    stack=True,
    label=[label for label in labels],
    color=[color for color in colors],
    ax=ax
    )

hep.histplot(
    bin_array_signal2,
    w2=bin_array_signal_errors2**2,
    histtype="step",
    #density = True,
    stack=False,
    label='2HDMa_1500_150_10',
    #color=[colors[sample] for sample in backgrounds],
    ax=ax
    )

ax.set_xlabel(r'$Bin$')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
ax.legend(prop={'size': 14})

fig.savefig('/home/users/ewallace/public_html/HbbMET/plots/signal_region/unrolled_stack_2000_750_alt.png')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(13,10) )
MT_vs_sdmass['2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1250_MH4_750_MH2_1250_MHC_1250']['central'].plot(counts=True, equidistant='xy', counts_formatter="{:.2e} \n$\pm$\n {:.2e}".format, counts_fontsize=10,)
ax.set_xlabel(r'$M_{T}$')
ax.set_ylabel(r'$softdrop\ mass$')
fig.savefig('/home/users/ewallace/public_html/HbbMET/plots/signal_region/MT_vs_sdmass_1250_750.png')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(13,10) )
MT_vs_sdmass['TT']['central'].plot(counts=True, equidistant='xy', counts_formatter="{:.2e} \n$\pm$\n {:.2e}".format, counts_fontsize=10,)
ax.set_xlabel(r'$M_{T}$')
ax.set_ylabel(r'$softdrop\ mass$')
fig.savefig('/home/users/ewallace/public_html/HbbMET/plots/signal_region/MT_vs_sdmass_tt.png')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(13,10) )
MT_vs_sdmass['WJetsToLNu_Njet']['central'].plot(counts=True, equidistant='xy', counts_formatter="{:.2e} \n$\pm$\n {:.2e}".format, counts_fontsize=10,)
ax.set_xlabel(r'$M_{T}$')
ax.set_ylabel(r'$softdrop\ mass$')
fig.savefig('/home/users/ewallace/public_html/HbbMET/plots/signal_region/MT_vs_sdmass_W.png')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(13,10) )
MT_vs_sdmass['QCD_bEnriched_HT']['central'].plot(counts=True, equidistant='xy', counts_formatter="{:.2e} \n$\pm$\n {:.2e}".format, counts_fontsize=10,)
ax.set_xlabel(r'$M_{T}$')
ax.set_ylabel(r'$softdrop\ mass$')
fig.savefig('/home/users/ewallace/public_html/HbbMET/plots/signal_region/MT_vs_sdmass_QCD.png')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(13,10) )
MT_vs_sdmass['ZJetsToNuNu_HT']['central'].plot(counts=True, equidistant='xy', counts_formatter="{:.2e} \n$\pm$\n {:.2e}".format, counts_fontsize=10,)
ax.set_xlabel(r'$M_{T}$')
ax.set_ylabel(r'$softdrop\ mass$')
fig.savefig('/home/users/ewallace/public_html/HbbMET/plots/signal_region/MT_vs_sdmass_Z.png')